# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities1.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kirensk,57.7853,108.1119,258.28,87,79,3.89,RU,1733728138
1,1,puerto natales,-51.7236,-72.4875,279.09,75,100,1.42,CL,1733728141
2,2,aasiaat,68.7098,-52.8699,257.92,87,52,3.09,GL,1733728142
3,3,ushuaia,-54.8000,-68.3000,275.71,97,100,2.64,AR,1733728143
4,4,waitangi,-43.9535,-176.5597,289.16,97,100,3.58,NZ,1733728144


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True, 
    size = "Humidity",
    scale= 1,
    color= "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 250) & (city_data_df["Max Temp"] > 20) & (city_data_df["Wind Speed"] < 6)
]

# Drop any rows with null values
narrowed_clean_df = narrowed_df.dropna()

# Display sample data
narrowed_clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,tiksi,71.6872,128.8694,245.96,99,82,2.77,RU,1733728195
77,77,bilibino,68.0546,166.4372,245.97,85,56,0.71,RU,1733728223
96,96,aykhal,66.0000,111.5000,240.40,100,100,2.62,RU,1733728243
140,140,udachny,66.4167,112.4000,238.75,100,98,2.04,RU,1733728296
186,186,mirnyy,62.5353,113.9611,241.87,100,74,1.80,RU,1733728348
202,202,mirny,62.5353,113.9611,241.87,100,74,1.80,RU,1733728366
216,216,borogontsy,62.6683,131.1683,240.13,100,96,1.66,RU,1733728381
320,320,chegdomyn,51.1167,133.0167,249.89,97,21,0.06,RU,1733728496
381,381,khandyga,62.6667,135.6000,242.61,100,95,0.90,RU,1733728561
420,420,vilyuysk,63.7553,121.6247,249.37,99,100,1.68,RU,1733728605


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
50,tiksi,RU,71.6872,128.8694,99,
77,bilibino,RU,68.0546,166.4372,85,
96,aykhal,RU,66.0000,111.5000,100,
140,udachny,RU,66.4167,112.4000,100,
186,mirnyy,RU,62.5353,113.9611,100,
202,mirny,RU,62.5353,113.9611,100,
216,borogontsy,RU,62.6683,131.1683,100,
320,chegdomyn,RU,51.1167,133.0167,97,
381,khandyga,RU,62.6667,135.6000,100,
420,vilyuysk,RU,63.7553,121.6247,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tiksi - nearest hotel: Арктика
bilibino - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
mirnyy - nearest hotel: No hotel found
mirny - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found
chegdomyn - nearest hotel: No hotel found
khandyga - nearest hotel: No hotel found
vilyuysk - nearest hotel: Мария
verkhnevilyuysk - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
50,tiksi,RU,71.6872,128.8694,99,Арктика
77,bilibino,RU,68.0546,166.4372,85,No hotel found
96,aykhal,RU,66.0000,111.5000,100,No hotel found
140,udachny,RU,66.4167,112.4000,100,No hotel found
186,mirnyy,RU,62.5353,113.9611,100,No hotel found
202,mirny,RU,62.5353,113.9611,100,No hotel found
216,borogontsy,RU,62.6683,131.1683,100,No hotel found
320,chegdomyn,RU,51.1167,133.0167,97,No hotel found
381,khandyga,RU,62.6667,135.6000,100,No hotel found
420,vilyuysk,RU,63.7553,121.6247,99,Мария


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)